Fine-tuning best T5 Transformer 🤖
-----------------------------------

In this notebook, we will continue the fine-tuning of T5 transformer on the new extracted sentences from the book **Grammaire de Wolof Moderne** without considering the definitions. We provide, bellow, the main evaluation figures, obtained from the hyperparameter search step. We will evaluate the training on the validation dataset.

- Parallel coordinates from panel:

- Parameter importance char: 
[t5_v3_importance](https://wandb.ai/oumar-kane-team/small-t5-cross-fw-translation-bayes-hpsearch-v3/reports/undefined-23-05-16-10-36-17---Vmlldzo0Mzc4NDY0?accessToken=eyaiyrid0qz1zg2jkq3fc65biw53084dpfitbi0dgonq6mweupw6kgjml9d2nv1w)

We can see in the above chart that the batch is the most important parameter with a negative correlation with the BLEU score (meaning that a lower batch size is better). Next, we the probability of modifying a character in the french corpus is also important and a high probability provide a better BLEU score.  

In [1]:
# let us import all necessary libraries
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, T5TokenizerFast, set_seed, AdamW, get_linear_schedule_with_warmup, T5ForConditionalGeneration,\
    get_cosine_schedule_with_warmup, Adafactor
from wolof_translate.utils.sent_transformers import TransformerSequences
from wolof_translate.utils.improvements.end_marks import add_end_mark # added
from torch.nn import TransformerEncoderLayer, TransformerDecoderLayer
from torch.utils.data import Dataset, DataLoader, random_split
from wolof_translate.data.dataset_v3 import SentenceDataset # v2 -> v3
from wolof_translate.utils.sent_corrections import *
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import _LRScheduler
# from custom_rnn.utils.kwargs import Kwargs
from torch.nn.utils.rnn import pad_sequence
from plotly.subplots import make_subplots
from nlpaug.augmenter import char as nac
from torch.utils.data import DataLoader
# from datasets  import load_metric # make pip install evaluate instead
# and pip install sacrebleu for instance
from torch.nn import functional as F
import plotly.graph_objects as go
from tokenizers import Tokenizer
import matplotlib.pyplot as plt
import pytorch_lightning as tl
from tqdm import tqdm, trange
from functools import partial
from torch.nn import utils
from copy import deepcopy
from torch import optim
from typing import *
from torch import nn
import pandas as pd
import numpy as np
import itertools
import evaluate
import random
import string
import shutil
import wandb
import torch
import json
import copy
import os

# set a global seed
tl.seed_everything(0)

os.environ["WANDB_DISABLED"] = "true"

c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 0


## French to wolof

### Configure dataset 🔠

In [2]:
# recuperate the tokenizer from a json file
tokenizer = T5TokenizerFast(tokenizer_file=f"wolof-translate/wolof_translate/tokenizers/t5_tokenizers/tokenizer_v3.json")


In [3]:
def recuperate_datasets(wf_char_p: float, wf_word_p: float, max_len: int, end_mark_opt: int):

  # Let us recuperate the end_mark adding option
  if end_mark_opt == 1:
    # Create augmentation to add on French sentences
    fr_augmentation_1 = TransformerSequences(nac.KeyboardAug(aug_char_p=wf_char_p, aug_word_p=wf_word_p,
                                                             aug_word_max = max_len),
                                          remove_mark_space, delete_guillemet_space)
    
    fr_augmentation_2 = TransformerSequences(remove_mark_space, delete_guillemet_space)

  else:
    
    if end_mark_opt == 2:

      end_mark_fn = partial(add_end_mark, end_mark_to_remove = '!', replace = True)
    
    elif end_mark_opt == 3:

      end_mark_fn = partial(add_end_mark)
    
    elif end_mark_opt == 4:

      end_mark_fn = partial(add_end_mark, end_mark_to_remove = '!')

    # Create augmentation to add on French sentences
    fr_augmentation_1 = TransformerSequences(nac.KeyboardAug(aug_char_p=wf_char_p, aug_word_p=wf_word_p,
                                                             aug_word_max = max_len),
                                          remove_mark_space, delete_guillemet_space, end_mark_fn)
    
    fr_augmentation_2 = TransformerSequences(remove_mark_space, delete_guillemet_space, end_mark_fn)

  # Recuperate the train dataset
  train_dataset_aug = SentenceDataset(f"data/extractions/new_data/train_set.csv",
                                        tokenizer,
                                        truncation = True, max_len=max_len,
                                        corpus_1 = 'wolof',
                                        corpus_2='french',
                                        cp1_transformer = fr_augmentation_1,
                                        cp2_transformer = fr_augmentation_2
                                        )

  # Recuperate the valid dataset
  valid_dataset = SentenceDataset(f"data/extractions/new_data/valid_set.csv",
                                        tokenizer, max_len=max_len,
                                        corpus_1='wolof',
                                        corpus_2='french',
                                        cp1_transformer = fr_augmentation_2,
                                        cp2_transformer = fr_augmentation_2,
                                        truncation = True)
  
  # Return the datasets
  return train_dataset_aug, valid_dataset

### Configure the model and the evaluation function ⚙️

Let us evaluate the predictions with the `bleu` metric.

In [4]:
%%writefile wolof-translate/wolof_translate/utils/evaluation.py
from tokenizers import Tokenizer
from typing import *
import numpy as np
import evaluate

class TranslationEvaluation:
    
    def __init__(self, 
                 tokenizer: Tokenizer,
                 decoder: Union[Callable, None] = None,
                 metric = evaluate.load('sacrebleu'),
                 ):
        
        self.tokenizer = tokenizer
        
        self.decoder = decoder
        
        self.metric = metric
    
    def postprocess_text(self, preds, labels):
        
        preds = [pred.strip() for pred in preds]
        
        labels = [[label.strip()] for label in labels]
        
        return preds, labels

    def compute_metrics(self, eval_preds):

        preds, labels = eval_preds

        if isinstance(preds, tuple):
        
            preds = preds[0]
        
        decoded_preds = self.tokenizer.batch_decode(preds, skip_special_tokens=True)

        labels = np.where(labels != -100, labels, self.tokenizer.pad_token_id)
        
        decoded_labels = self.tokenizer.batch_decode(labels, skip_special_tokens=True)

        decoded_preds, decoded_labels = self.postprocess_text(decoded_preds, decoded_labels)

        result = self.metric.compute(predictions=decoded_preds, references=decoded_labels)
        
        result = {"bleu": result["score"]}

        prediction_lens = [np.count_nonzero(pred != self.tokenizer.pad_token_id) for pred in preds]
        
        result["gen_len"] = np.mean(prediction_lens)
        
        result = {k: round(v, 4) for k, v in result.items()}
        
        return result

Overwriting wolof-translate/wolof_translate/utils/evaluation.py


Let us initialize the evaluation object.

In [5]:
%run wolof-translate/wolof_translate/utils/evaluation.py
evaluation = TranslationEvaluation(tokenizer)


### Searching for the best parameters 🕖

In [6]:
from wolof_translate.models.transformers.optimization import TransformerScheduler
from wolof_translate.trainers.transformer_trainer import ModelRunner
from wolof_translate.utils.evaluation import TranslationEvaluation
from wolof_translate.models.transformers.main import Transformer
from wolof_translate.utils.split_with_valid import split_data


-------------

### --- Wandb V3

In [18]:
# let us initialize the hyperparameter configuration 
config = {
    'random_state': 0,
    'fr_char_p': 0.05749357920504272,
    'fr_word_p': 0.22418236350190135,
    'learning_rate': 0.006729837552295897,
    'weight_decay': 0.030529270716030843,
    'batch_size': 32,
    'warmup_ratio': 0.0,
    'max_epoch': 507,
    'epochs': 507,
    'max_len': 51,
    'end_mark': 4,
    'bleu': 5.3427,
    'model_dir': 'data/checkpoints/wf_t5_small_custom_train_v3_checkpoints/',
    'new_model_dir': 'data/checkpoints/t5_small_custom_train_results_wf_v3/'
}

# Initialize the model name
model_name = 't5-small'

# import the model with its pre-trained weights
model = T5ForConditionalGeneration.from_pretrained(model_name)

# resize the token embeddings
model.resize_token_embeddings(len(tokenizer))

# let us initialize the evaluation class
evaluation = TranslationEvaluation(tokenizer)

# let us initialize the trainer
trainer = ModelRunner(model, seed = 0, version = 3, evaluation = evaluation, optimizer = Adafactor)

# split the data
split_data(config['random_state'])

# recuperate train and test set
train_dataset, test_dataset = recuperate_datasets(config['fr_char_p'], 
                                                    config['fr_word_p'], config['max_len'],
                                                    config['end_mark'])

# let us calculate the appropriate warmup steps (let us take a max epoch of 100)
length = len(train_dataset)

n_steps = length // config['batch_size']

num_steps = config['max_epoch'] * n_steps

warmup_steps = (config['max_epoch'] * n_steps) * config['warmup_ratio']

# Initialize the scheduler parameters
scheduler_args = {'num_warmup_steps': warmup_steps, 'num_training_steps': num_steps}

# Initialize the optimizer parameters
optimizer_args = {
    'lr': config['learning_rate'],
    'weight_decay': config['weight_decay'],
    # 'betas': (0.9, 0.98),
    'relative_step': False
}

# Initialize the loaders parameters
train_loader_args = {'batch_size': config['batch_size']}

# Add the datasets and hyperparameters to trainer
trainer.compile(train_dataset, test_dataset, tokenizer, train_loader_args,
                optimizer_kwargs = optimizer_args,
                # lr_scheduler=get_linear_schedule_with_warmup,
                lr_scheduler=get_cosine_schedule_with_warmup,
                lr_scheduler_kwargs=scheduler_args, 
                predict_with_generate = True,
                hugging_face = True,
                logging_dir="data/logs/t5_small_custom_train_wf"
                )

# We will from checkpoints so let us the model
# trainer.load(config['model_dir'], load_best=True) # Only for the first loading
trainer.load(config['new_model_dir'], load_best=True)

        

### ---

In [8]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

  0%|          | 0/195 [00:00<?, ?it/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\torch\optim\lr_scheduler.py:257: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


For epoch 6: 
{Learning rate: [0.006661809115500007]}


Test batch number 1:   0%|          | 0/10 [00:00<?, ?batches/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Test batch number 10: 100%|██████████| 10/10 [00:03<00:00,  2.93batches/s]



Metrics: {'train_loss': 0.29751730364997214, 'test_loss': 0.4935239255428314, 'bleu': 5.0221, 'gen_len': 9.0753}




  1%|          | 1/195 [00:25<1:23:16, 25.75s/it]

...

 99%|█████████▉| 194/195 [1:12:26<00:21, 21.76s/it]

For epoch 200: 
{Learning rate: [0.004022305767819456]}


Test batch number 10: 100%|██████████| 10/10 [00:03<00:00,  2.66batches/s]



Metrics: {'train_loss': 0.005946798658952481, 'test_loss': 0.5689922019839286, 'bleu': 22.0488, 'gen_len': 10.1507}




100%|██████████| 195/195 [1:12:48<00:00, 22.40s/it]


### ---

In [13]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

  0%|          | 0/53 [00:00<?, ?it/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\torch\optim\lr_scheduler.py:257: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


For epoch 201: 
{Learning rate: [0.004008700080460279]}


Test batch number 1:   0%|          | 0/10 [00:00<?, ?batches/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Test batch number 10: 100%|██████████| 10/10 [00:03<00:00,  3.05batches/s]



Metrics: {'train_loss': 0.004984950828479558, 'test_loss': 0.5810729935765266, 'bleu': 22.141, 'gen_len': 10.1301}




  2%|▏         | 1/53 [00:18<15:43, 18.15s/it]

...

 98%|█████████▊| 52/53 [18:39<00:22, 22.36s/it]

For epoch 253: 
{Learning rate: [0.0033012043377830167]}


Test batch number 10: 100%|██████████| 10/10 [00:04<00:00,  2.29batches/s]



Metrics: {'train_loss': 0.0043112455765012565, 'test_loss': 0.5758065916597843, 'bleu': 21.3934, 'gen_len': 9.9178}




100%|██████████| 53/53 [19:02<00:00, 21.55s/it]


### ---

In [15]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

  0%|          | 0/47 [00:00<?, ?it/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\torch\optim\lr_scheduler.py:257: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


For epoch 254: 
{Learning rate: [0.0032434909765787574]}


Test batch number 1:   0%|          | 0/10 [00:00<?, ?batches/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Test batch number 10: 100%|██████████| 10/10 [00:03<00:00,  3.15batches/s]



Metrics: {'train_loss': 0.0045043093169380615, 'test_loss': 0.5777980253100395, 'bleu': 21.1843, 'gen_len': 9.9658}




  2%|▏         | 1/47 [00:17<13:43, 17.89s/it]

...

 98%|█████████▊| 46/47 [17:06<00:22, 22.54s/it]

For epoch 300: 
{Learning rate: [0.002280093238049504]}


Test batch number 10: 100%|██████████| 10/10 [00:03<00:00,  2.63batches/s]



Metrics: {'train_loss': 0.004264040597396471, 'test_loss': 0.5849293962121009, 'bleu': 22.3968, 'gen_len': 10.1507}




100%|██████████| 47/47 [17:27<00:00, 22.30s/it]


### ---

In [ ]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

  0%|          | 0/207 [00:00<?, ?it/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\torch\optim\lr_scheduler.py:257: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


For epoch 301: 
{Learning rate: [0.0022598846152403727]}


Test batch number 1:   0%|          | 0/10 [00:00<?, ?batches/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Test batch number 10: 100%|██████████| 10/10 [00:03<00:00,  3.15batches/s]



Metrics: {'train_loss': 0.003966887626319942, 'test_loss': 0.592028059065342, 'bleu': 21.7206, 'gen_len': 10.0753}




  0%|          | 1/207 [00:17<1:00:05, 17.50s/it]

...

100%|█████████▉| 206/207 [1:13:54<00:21, 21.07s/it]

For epoch 507: 
{Learning rate: [8.76378390630555e-06]}


Test batch number 10: 100%|██████████| 10/10 [00:03<00:00,  2.76batches/s]



Metrics: {'train_loss': 0.0032587808517103153, 'test_loss': 0.5891663879156113, 'bleu': 21.6998, 'gen_len': 10.3151}




100%|██████████| 207/207 [1:14:15<00:00, 21.52s/it]


### --- Wandb V3_2


In [7]:
# let us initialize the hyperparameter configuration 
config = {
    'random_state': 0,
    'fr_char_p': 0.7707750109071422,
    'fr_word_p': 0.07546630694952816,
    'learning_rate': 0.00857398866833631,
    'weight_decay': 0.015767393515757393,
    'batch_size': 16,
    'warmup_ratio': 0.0,
    'max_epoch': 827,
    'epochs': 600,
    'mid_epoch': 413,
    'max_len': 15,
    'end_mark': 4,
    'bleu': 8.4756,
    'model_dir': 'data/checkpoints/wf_t5_small_custom_train_v3_2_checkpoints/',
    'new_model_dir': 'data/checkpoints/t5_small_custom_train_results_wf_v3_2/'
}

# Initialize the model name
model_name = 't5-small'

# import the model with its pre-trained weights
model = T5ForConditionalGeneration.from_pretrained(model_name)

# resize the token embeddings
model.resize_token_embeddings(len(tokenizer))

# let us initialize the evaluation class
evaluation = TranslationEvaluation(tokenizer)

# let us initialize the trainer
trainer = ModelRunner(model, seed = 0, version = '3_2', evaluation = evaluation, optimizer = Adafactor)

# split the data
split_data(config['random_state'])

# recuperate train and test set
train_dataset, test_dataset = recuperate_datasets(config['fr_char_p'], 
                                                    config['fr_word_p'], config['max_len'],
                                                    config['end_mark'])

# let us calculate the appropriate warmup steps (let us take a max epoch of 100)
length = len(train_dataset)

n_steps = length // config['batch_size']

num_steps = config['max_epoch'] * n_steps

warmup_steps = (config['max_epoch'] * n_steps) * config['warmup_ratio']

# Initialize the scheduler parameters
scheduler_args = {'num_warmup_steps': warmup_steps, 'num_training_steps': num_steps}

# Initialize the optimizer parameters
optimizer_args = {
    'lr': config['learning_rate'],
    'weight_decay': config['weight_decay'],
    # 'betas': (0.9, 0.98),
    'relative_step': False
}

# Initialize the loaders parameters
train_loader_args = {'batch_size': config['batch_size']}

# Add the datasets and hyperparameters to trainer
trainer.compile(train_dataset, test_dataset, tokenizer, train_loader_args,
                optimizer_kwargs = optimizer_args,
                # lr_scheduler=get_linear_schedule_with_warmup,
                lr_scheduler=get_cosine_schedule_with_warmup,
                lr_scheduler_kwargs=scheduler_args, 
                predict_with_generate = True,
                hugging_face = True,
                logging_dir="data/logs/t5_small_custom_train_wf"
                )

# We will from checkpoints so let us the model
# trainer.load(config['model_dir'], load_best=True) # Only for the first loading
trainer.load(config['new_model_dir'], load_best = True)

        

### ---

In [9]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

  0%|          | 0/195 [00:00<?, ?it/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\torch\optim\lr_scheduler.py:257: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


For epoch 6: 
{Learning rate: [0.008521510794211212]}


Test batch number 1:   0%|          | 0/10 [00:00<?, ?batches/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Test batch number 10: 100%|██████████| 10/10 [00:03<00:00,  2.64batches/s]



Metrics: {'train_loss': 0.6426358139369546, 'test_loss': 1.7641808331012725, 'bleu': 9.6324, 'gen_len': 10.5411}




  1%|          | 1/195 [00:55<3:00:00, 55.67s/it]

...

 99%|█████████▉| 194/195 [1:18:47<00:24, 24.53s/it]

For epoch 200: 
{Learning rate: [0.00648536927815744]}


Test batch number 10: 100%|██████████| 10/10 [00:03<00:00,  3.07batches/s]



Metrics: {'train_loss': 0.024924431319870963, 'test_loss': 1.980709195137024, 'bleu': 23.4599, 'gen_len': 9.7877}




100%|██████████| 195/195 [1:19:12<00:00, 24.37s/it]


### ---

In [13]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

  0%|          | 0/213 [00:00<?, ?it/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\torch\optim\lr_scheduler.py:257: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


For epoch 201: 
{Learning rate: [0.006474873703332421]}


Test batch number 1:   0%|          | 0/10 [00:00<?, ?batches/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Test batch number 10: 100%|██████████| 10/10 [00:03<00:00,  2.59batches/s]



Metrics: {'train_loss': 0.022560643984811245, 'test_loss': 2.006405603885651, 'bleu': 24.3427, 'gen_len': 9.8288}




  0%|          | 1/213 [00:23<1:22:09, 23.25s/it]

...

100%|█████████▉| 212/213 [1:23:36<00:24, 24.00s/it]

For epoch 413: 
{Learning rate: [0.004249811840428299]}


Test batch number 10: 100%|██████████| 10/10 [00:03<00:00,  2.72batches/s]



Metrics: {'train_loss': 0.0166580138994926, 'test_loss': 2.001829054951668, 'bleu': 22.2644, 'gen_len': 10.1438}




100%|██████████| 213/213 [1:24:00<00:00, 23.67s/it]


### ---

In [15]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

  0%|          | 0/187 [00:00<?, ?it/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\torch\optim\lr_scheduler.py:257: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


For epoch 414: 
{Learning rate: [0.004212105608556713]}


Test batch number 1:   0%|          | 0/10 [00:00<?, ?batches/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Test batch number 10: 100%|██████████| 10/10 [00:02<00:00,  3.49batches/s]



Metrics: {'train_loss': 0.015629152222018314, 'test_loss': 2.0144938349723818, 'bleu': 22.301, 'gen_len': 9.7329}




  1%|          | 1/187 [00:22<1:08:17, 22.03s/it]

...

 99%|█████████▉| 186/187 [1:13:44<00:20, 20.97s/it]

For epoch 600: 
{Learning rate: [0.0014193019120657582]}


Test batch number 10: 100%|██████████| 10/10 [00:02<00:00,  3.44batches/s]



Metrics: {'train_loss': 0.012563905811608803, 'test_loss': 2.0592204332351685, 'bleu': 24.4999, 'gen_len': 9.9041}




100%|██████████| 187/187 [1:14:05<00:00, 23.77s/it]


### Predictions and Evaluation

In [8]:
# initialize the transformation sequence
end_mark_fn = partial(add_end_mark)
fr_augmentation = TransformerSequences(remove_mark_space, delete_guillemet_space, end_mark_fn)

# let us get the test set
test_dataset = SentenceDataset(f"data/extractions/new_data/test_set.csv",
                                        corpus_1='wolof',
                                        corpus_2='french',
                                        tokenizer = tokenizer,
                                        cp1_transformer = fr_augmentation,
                                        cp2_transformer = fr_augmentation,
                                        truncation = True)

Let us make the evaluation and print the predicted sentences.

In [9]:
# evaluation with test set
df_ft_to_wf = trainer.evaluate(test_dataset)

Evaluation batch number 1:   0%|          | 0/11 [00:00<?, ?batches/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Evaluation batch number 11: 100%|██████████| 11/11 [00:08<00:00,  1.27batches/s]


In [10]:
df_ft_to_wf[1].tail(10)

,original_sentences,translations,predictions
152,"Yaa ŋgi, dem ŋga.","Te voila, tu as été.","Toi, tu as été."
153,Bëgg na ŋga dem.,Il veut que tu viennes.,Il veut que tu viennes.
154,Liggéeykat yi man ag yaw la.,Les travailleurs c'est toi et moi.,"C'est toi qui l'as vu, lui, Samba."
155,Foofee fan?,Où?,Tu n'y avais vu personne?
156,"Yaa ŋgi, mi ŋgi.","Te voilà, le voilà.","Toi, tu es là."
157,Gis ŋga kooku?,Tu as vu celui-ci?,Tu as vu celui-là?
158,Dem naa ba ci moom.,J'ai été jusqu'à lui.,J'ai été jusqu'à Saint-Louis.
159,Yéen ñan la wax?,Il parle de vous?,Il parle desquelles de vous?
160,Moo doon ganam.,C'était son hôte habituellement.,C'est lui qui n'eût pas été.
161,Nil waa ji na ñëw.,Dis à la personne qu'elle vienne.,Que la personne sorte est venu.


In [11]:
# let us display 100 samples
pd.options.display.max_rows = 100
df_ft_to_wf[1].sample(100)

,original_sentences,translations,predictions
66,Yaa daan ganu Mustaf.,Tu étais d'habitude l'hôte de Mustapha.,C'est toi qui es un autre.
131,Génnéel mépp xar moo gis!,Fais sortir tout mouton que tu vois!,Tout autre mouton que tu vois.
145,Kooku dem ku më bëgg la!,"Celui qui est parti, c'est quelqu'un que j'app...","C'est quelqu'un que j'apprécie, celui qui est ..."
7,Gis naa sana xarit yi!,J'ai vu mes amis!,J'ai vu tes autres amis.
44,Gis naa nagu Tugal.,J'ai vu des vaches d'Europe.,J'ai vu vaches d'Europe.
74,"Soo demee, kookule la!","C'est celui-là, peut-être!",C'est celui-là peut-être.
124,Noonu mu demoon foofa.,Comme il avait ainsi été en ce lieu.,Alors il partit.
159,Yéen ñan la wax?,Il parle de vous?,Il parle desquelles de vous?
86,Su dulwoon Lawbe.,S'il n'était pas Laobe.,"Quand on ira, tout ira."
122,Noonu la woon.,C'était ainsi.,Ainsi fut.
